# Selection et entrainement des modèles 

***Résumé Exécutif du Feature Engineering - Prévision Solaire J+1***

L'étape de feature_engineering a consisté à la prise en compte des conclusions de l'analyse exploratoire (EDA), ainsi que la création de garde-fous données manquantes/outliers ainsi que la création de features cycliques et de variables laggées.

**Données** : Dataset horaire sur 2.5 ans (01/2023 - 05/2025) de production RTE et prévisions OpenMétéo. Identification de 2 valeurs manquantes nocturnes dans la cible, qui seront imputées à 0.

**Gardes-fous :**
- Création de tests IQR/Z-score et raise des outliers en intersection des deux filtres ;
- Interpolation des séquences temporelles inférieures à 3h consécutives ;
- Lors d'une absence de séquence de plus de 3 heures, création d'un reporting des séquences les plus longues, et potentiellement création d'un futur algorithme KNN - Filtre de Kalman.

**Features créées :**
- Création de features cycliques heures + mois, en fonction de la saisonnalité du cycle solaire ;
- Création de features laggées (data leakage évité): 
  - Retard de 24, 32 et 48 (observation des cross-correlation + cohérent physiquement) ;
  - Moyennes mobiles de 24, 32 et 48 périodes (analogue aux features retard).
 
**Etapes effectuées dans ce notebook** :

- Transformations statistiques pour les données LSTM et/ou SARIMAX si nécessaire ;
- Baseline SARIMAX avec les features physiques les plus corrélées ;
- Sélection de features (Embedding via LightGBM) ;
- Validation croisée "Expanding Window" avec optimisation des hyperparamètres (Optuna) ;
- Développement de modèles LightGBM et LSTM, avec MC dropout et regression quantile pour quantifier l'incertitude des modèles.

### Import des librairies

In [7]:
# Base
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional, Dict
import optuna
import traceback
import logging
import warnings

# Modèles
from statsmodels.tsa.statespace.sarimax import SARIMAX
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import torch.nn as nn
import torch

# Solaire
from astral import LocationInfo
from astral.sun import sun

In [9]:
# Confirmation d'être à la racine du dossier
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(project_root)

## Préparation des jeux de validation croisée

La validation croisée (CV) est effectuée avec la validation croisée imbriquée (nested CV). Pour l'optimisation des hyperparamètres dans chaque fold, nous utiliserons l'alogrithme TPE supporté par Optuna. Du pruning sera effectué sur l'entrainement du LSTM Seq2seq.

In [10]:
df = pd.read_csv("data/processed/df_engineered.csv", index_col=0)
df_power = pd.read_csv("data/processed/occitanie_installed_power.csv", index_col=0)
df.index = pd.to_datetime(df.index, utc=True).tz_convert("Europe/Paris")
df_power.index = pd.to_datetime(df_power.index, utc=True).tz_convert("Europe/Paris")

In [11]:
def solar_power_norm(df_hourly_puiss: pd.DataFrame, 
                     df_installed_power: pd.DataFrame,
                     col_hourly_puiss: str, 
                     col_installed_power: str) -> pd.Series:
    
    """Prends en entrée la puissance produite horaire (df_hourly_puiss), et la normalise par la capacité régionale de production solaire (df_installed_power)

    Args:
        df_hourly_puiss (pd.DataFrame): DataFrame contenant les puissances solaires produites à la maille horaire
        df_installed_power (pd.DataFrame): DataFrame contenant la capacité régionale solaire  à la maille horaire
        col_daily_puiss (str): Nom de la colonne des puissances solaires
        col_installed_power (str): Nom de la colonne de la capacité régionale solaire

    Returns:
        pd.Series: Colonne contenant les puissances solaires normalisées par la capacité régionale
    """
    # if len(df_hourly_puiss) != len(df_installed_power):
    #     raise ValueError(f"Les deux dataframes n'ont pas la même longueur"
    #                      f" {len(df_hourly_puiss)} vs {len(df_installed_power)}")
    
    return df_hourly_puiss[col_hourly_puiss]/df_installed_power[col_installed_power]

In [12]:
def prepare_data_for_nn(df: pd.DataFrame, target_col: str, features_to_use: Optional[list[str]] = None):
    """Retourne pour dataframe df les données d'entrée du modèle X transformée par MinMaxScaler (X_scaled) et la colonne cible (target_col)

    Args:
        df (pd.DataFrame): DataFrame df avec toutes les colonnes, colonne cible comprise.
        target_col (str): Colonne cible.
        features_to_use (Optional[list[str]], optional): Features à utiliser (optionnel). Defaults to None.

    Returns:
        X_scaled (pd.DataFrame), y (pd.Series) : DataFrame scalé et colonne cible.
    """
    if features_to_use:
        X = df[features_to_use]
    else:
        X = df.drop(columns=target_col)
    y = df[target_col]

    # Scaling des features
    scaler = MinMaxScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), index = X.index, columns=X.columns)

    return X_scaled, y

In [13]:
def time_series_fold(X: pd.DataFrame, y: pd.Series):
    """
    Génère des splits temporels (TimeSeriesSplit) en conservant les index.
    
    Args:
        X (pd.DataFrame): Variables explicatives.
        y (pd.Series): Variable cible.
    
    Returns:
        dict_ts_fold (dict): Dictionnaire contenant les folds de train/test.
    """
    dict_ts_fold = {}
    tscv = TimeSeriesSplit(gap=0, n_splits=5)

    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        dict_ts_fold[f'train_{i}'] = [X_train, y_train]
        dict_ts_fold[f'test_{i}'] = [X_test, y_test]
    
    return dict_ts_fold

In [14]:
def outer_time_series_fold(X: pd.DataFrame, y: pd.Series, outer_n_splits: int = 5):
    """Effectue une validation croisée externe avec TimeSeriesSplit"""

    dict_outer_fold = {}
    tscv = TimeSeriesSplit(gap=0, n_splits=outer_n_splits)

    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index] 
        dict_outer_fold[f'train_{i}'] = [X_train, y_train]
        dict_outer_fold[f'val_{i}'] = [X_test, y_test]
    
    return dict_outer_fold

In [15]:
def inner_time_series_fold(folds: dict, inner_n_splits : int = 3):
    """Effectue une validation croisée interne (nested CV) sur les folds d'entraînement"""
    
    dict_inner_fold = {}
    tscv = TimeSeriesSplit(gap=0, n_splits=inner_n_splits)

    for key, (X_train_outer, y_train_outer) in folds.items():
        
        if key.startswith("train"):
            
            for inner_i, (train_index, test_index) in enumerate(tscv.split(X_train_outer)):
                X_train_inner, X_val_inner = X_train_outer.iloc[train_index], X_train_outer.iloc[test_index]
                y_train_inner, y_val_inner = y_train_outer.iloc[train_index], y_train_outer.iloc[test_index]
                dict_inner_fold[f'{key}_inner_train_{inner_i}'] = [X_train_inner, y_train_inner]
                dict_inner_fold[f'{key}_inner_val_{inner_i}'] = [X_val_inner, y_val_inner]
    
    return dict_inner_fold

### Baseline SARIMAX

Les features choisies pour la baseline SARIMAX sont celles ayant un impact physique sur la production.

In [16]:
def get_sun_data_for_dates(observer, start_date_ts, end_date_ts):
    """
    Génère un DatetimeIndex pour toutes les heures entre l'aube et le crépuscule
    pour chaque jour d'une période donnée.
    """
    # Créer une plage de dates pour tous les jours de la période
    dates = pd.date_range(start=start_date_ts, end=end_date_ts, freq="D", tz="Europe/Paris").date

    # Initialiser l'index combiné
    combined_index = pd.DatetimeIndex([], tz="Europe/Paris")

    for day in dates:
        s = sun(observer, date=day)
        dawn_local = pd.Timestamp(s['dawn']).tz_convert("Europe/Paris")
        dusk_local = pd.Timestamp(s['dusk']).tz_convert("Europe/Paris")
        temp_range = pd.date_range(start=dawn_local, end=dusk_local, freq="h").floor("h")
        combined_index = combined_index.union(temp_range)

    return combined_index

In [17]:
# Feature physique
city = LocationInfo(name="Central", region="France", timezone="Europe/Paris", latitude=43.62476, longitude=2.34041)
print((
    f"Information for {city.name}/{city.region}\n"
    f"Timezone: {city.timezone}\n"
    f"Latitude: {city.latitude:.02f}; Longitude: {city.longitude:.02f}\n"
))
start_ts = df_power.index.min().floor("d")
end_ts = df_power.index.max().floor("d")

# is_day
index_time = get_sun_data_for_dates(city.observer, start_ts, end_ts)
index_df = pd.DataFrame(index=index_time)
index_df["is_day"] = 1

Information for Central/France
Timezone: Europe/Paris
Latitude: 43.62; Longitude: 2.34



In [18]:
feature_sarimax = ['temperature_2m', 'relative_humidity_2m', 'precipitation',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m',
       'wind_direction_10m', 'global_tilted_irradiance',
       'global_tilted_irradiance_delta_minmax', 'global_tilted_irradiance_std'] 
col_hourly_puiss = "solar_mw"
col_installed_power = "chronique_capacity"
solar_mw_normalized = solar_power_norm(df, df_power, col_hourly_puiss, col_installed_power)
solar_mw_normalized = solar_mw_normalized.dropna()

In [19]:
def train_sarimax(X_train: pd.DataFrame, 
                   y_train: pd.Series, 
                   X_val: pd.DataFrame, 
                   y_val: pd.Series, 
                   num_trials: Optional[int|None]) -> tuple:
    
    """Entrainement d'un modèle SARIMAX avec nombre d'essais pour optimisation.
    Retourne la quantification de son erreur (best_rmse) et ses hyperparamètres 
    optimaux (dict_best_params) sur un dataset de validation (X_val, y_val)

    Args:
        X_train (pd.DataFrame): Dataset d'entrainement
        y_train (pd.Series): Variable cible d'entrainement
        X_val (pd.DataFrame): Dataset de validation
        y_val (pd.Series): Variable cible de validation

    Returns:
        best_rmse (float), dict_best_params (Dict) : Erreur (best_rmse) 
        et ses hyperparamètres optimaux (dict_best_params)
    """
    # Initialisation
    best_rmse = np.inf
    dict_best_params = {}
    
    def objective_sarimax(trial) -> np.float64 :
        """Prend en entrée un set d'hyperparamètres SARIMAX issus du sampler d'Optuna, 
        et retourne le RMSE associé.

        Args:
            trial : Set d'hyperparamètres SARIMAX

        Returns:
            rmse (np.float64): Racine carrée de l'erreur quadratique moyenne du modèle
        """
        # HP
        d = 0
        s = 24

        order = (trial.suggest_int("p", 0, 3), 
                d, 
                trial.suggest_int("q", 0, 3))
        
        seasonal_order = (trial.suggest_int("P", 0, 2), 
                          trial.suggest_int("D", 0, 1), 
                          trial.suggest_int("D", 0, 1), s)

        # Model
        model = SARIMAX(endog=y_train, 
                            exog=X_train, 
                            order = order, 
                            seasonal_order = seasonal_order)
        
        # Training
        try:
            fitted_model = model.fit(disp=False, maxiter=100)
            predictions = fitted_model.get_prediction(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, predictions))
        
        except Exception as e:
            logging.info("Echec de l'entrainement du modèle%s", e)
            logging.debug("Détails complets :\n%s", traceback.format_exc())
            raise

        return rmse


    # Recherche des HP et prédictions
    try:
        study = optuna.create_study(direction="minimize")
        study.optimize(objective_sarimax, n_trials=num_trials, n_jobs=4)
    
    except Exception as e:
        logging.error(
        "Échec de la recherche d’HP. Paramètres : n_trials=%d, n_jobs=%d. Erreur : %s",
        num_trials, 4, str(e),
        exc_info=True)
        raise
    
    # Set optimal score/HP
    dict_best_params = study.best_params
    best_rmse = study.best_value

    return best_rmse, dict_best_params

In [20]:
def inner_cv_sarimax(outer_id: int, inner_folds: Dict, num_trials: int, inner_n_splits: int) -> Dict:
    
    """Renvoie la liste optimale d'HP en effectuant une cross_validation interne 
    sur le fold (outer_id), avec un nombre d'essais (num_trials)

    Args:
        outer_id (int): ID du fold externe étudié
        inner_folds (Dict): Folds internes
        num_trials (int): Nombre d'essais Optuna
        inner_n_split (int): Nombre de split dans tes outer splits

    Returns:
        fold_best_params (Dict) : Liste optimale d'HP pour le fold outer_id
    """

    #Initialisation
    inner_scores = []
    inner_params = []
    n_folds = inner_n_splits

    # CV interne
    for inner_id in range(n_folds):
        X_train_inner, y_train_inner = inner_folds[f'train_{outer_id}_inner_train_{inner_id}']
        X_val_inner, y_val_inner = inner_folds[f'train_{outer_id}_inner_val_{inner_id}']

        best_rmse, best_params = train_sarimax(X_train=X_train_inner, 
                                                y_train=y_train_inner, 
                                                X_val=X_val_inner, 
                                                y_val=y_val_inner,
                                                num_trials=num_trials)
        # Scoring sur la CV interne
        inner_scores.append(best_rmse)
        inner_params.append(best_params)

    # HP optimaux pour le outer fold concerné
    best_inner_idx = np.argmin(inner_scores)
    fold_best_params = inner_params[best_inner_idx]

    return fold_best_params

In [21]:
def nested_cv_sarimax(X: pd.DataFrame,
                      y: pd.Series,
                      num_trials: int,
                      outer_n_splits: int = 5,
                      inner_n_splits: int = 3) -> tuple:
    
    """Renvoie le RMSE moyen de l'ensemble des folds sur un modèle SARIMAX,
    avec recherche d'hyperparamètres par nested CV.

    Args:
        X (pd.DataFrame): Variables explicatives
        y (pd.Series): Variable cible
        num_trials (int): Nombre d'essais Optuna pour l'optimisation
        outer_n_splits (int): Nombre de splits externes
        inner_n_splits (int): Nombre de splits internes

    Returns:
        tuple: (outer_scores, mean_outer_score)
            - outer_scores : Liste des RMSE par fold externe
            - mean_outer_score : Moyenne des RMSE externes
    """
    # Création des folds externes et internes
    outer_folds = outer_time_series_fold(X, y, outer_n_splits=outer_n_splits)
    inner_folds = inner_time_series_fold(outer_folds, inner_n_splits=inner_n_splits)
    outer_scores = []

    for outer_id in range(outer_n_splits):
        
        # Séparation train/val externe
        X_train_outer, y_train_outer = outer_folds[f"train_{outer_id}"]
        X_val_outer, y_val_outer = outer_folds[f"val_{outer_id}"]

        # Sélection des meilleurs hyperparamètres via inner CV
        fold_best_params = inner_cv_sarimax(
            outer_id=outer_id,
            inner_folds=inner_folds,
            num_trials=num_trials,
            inner_n_splits=inner_n_splits
        )

        # Instanciation du meilleur modèle SARIMAX
        best_model = SARIMAX(
            endog=y_train_outer,
            exog=X_train_outer,
            order=fold_best_params["order"],
            seasonal_order=fold_best_params.get("seasonal_order", (0, 0, 0, 0)),
        )

        # Fit du modèle
        fitted_model = best_model.fit(disp=False)

        # Prédiction sur la validation externe
        y_pred_outer = fitted_model.get_prediction(
            start=y_val_outer.index[0],
            end=y_val_outer.index[-1],
            exog=X_val_outer
        )

        # Calcul du RMSE externe
        outer_rmse = np.sqrt(mean_squared_error(y_val_outer, y_pred_outer))
        outer_scores.append(outer_rmse)

    # Moyenne des scores externes
    mean_outer_score = np.mean(outer_scores)
    logging.info(f'RMSE global moyen attendu en production : {mean_outer_score}')

    return outer_scores, mean_outer_score


In [ ]:
# Normalization
X = df[feature_sarimax]
X = pd.merge(X, index_df, left_index=True, right_index=True, how="left").fillna(0)
normalized_solar_ts = df["solar_mw"]/df_power["chronique_capacity"]
normalized_solar_ts = normalized_solar_ts.dropna()
y = normalized_solar_ts.copy()

# Nested CV
outer_n_splits = 5
inner_n_splits = 2
num_trials = 10
outer_scores, mean_outer_score = nested_cv_sarimax(X=X, 
                                                    y=y, 
                                                    num_trials=num_trials, 
                                                    outer_n_splits=outer_n_splits, 
                                                    inner_n_splits=inner_n_splits)

[I 2025-08-18 21:49:23,178] A new study created in memory with name: no-name-fd665e5c-95f3-4ae9-ab00-6f3d7a8996db
d:\Anaconda\envs\solar_forecasting\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
d:\Anaconda\envs\solar_forecasting\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
d:\Anaconda\envs\solar_forecasting\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
d:\Anaconda\envs\solar_forecasting\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
d:\Anaconda\envs\solar_for

### Modèle LightGBM

Le modèle LightGBM est un modèle par arbre qui est particulièrement intéressant dans le cadre de la compréhension de données bruitées et des interactions non linéaires. Nous pourrions aussi le comparer à XGBoost, qui a la faculté d'éviter plus souvent l'overfitting (à plus long terme). Le RMSE sera pris comme métrique de minimisation pour permettre une meilleure prise en compte des évènements extrêmes. Le MAE, MAPE et SMAPE seront aussi retournés, cette fois à titre indicatif pour comparaison.

In [ ]:
def train_lightgbm(X_train: pd.DataFrame, 
                   y_train: pd.Series, 
                   X_val: pd.DataFrame, 
                   y_val: pd.Series, 
                   num_trials: Optional[int|None]) -> tuple:
    
    """Entrainement d'un modèle LightGBM avec nombre d'essais pour optimisation.
    Retourne la quantification de son erreur (best_rmse) et ses hyperparamètres 
    optimaux (dict_best_params) sur un dataset de validation (X_val, y_val)

    Args:
        X_train (pd.DataFrame): Dataset d'entrainement
        y_train (pd.Series): Variable cible d'entrainement
        X_val (pd.DataFrame): Dataset de validation
        y_val (pd.Series): Variable cible de validation

    Returns:
        best_rmse (float), dict_best_params (Dict) : Erreur (best_rmse) 
        et ses hyperparamètres optimaux (dict_best_params)
    """
    # Initialisation
    best_rmse = np.inf
    dict_best_params = {}
    
    def objective_lightgbm(trial) -> np.float64 :
        """Prend en entrée un set d'hyperparamètres LightGBM issus du sampler d'Optuna, 
        et retourne le RMSE associé.

        Args:
            trial : Set d'hyperparamètres LightGBM

        Returns:
            rmse (np.float64): Racine carrée de l'erreur quadratique moyenne du modèle
        """
        # HP
        num_leaves = trial.suggest_int("num_leaves", 10, 100)
        max_depth = trial.suggest_int("max_depth", 3, 15)
        learning_rate = trial.suggest_float("learning_rate", 0.005, 0.2, log=True)
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        min_child_samples = trial.suggest_int("min_child_samples", 10, 50)

        # Model
        model = lgb.LGBMRegressor(num_leaves=num_leaves, 
                                max_depth=max_depth, 
                                learning_rate=learning_rate,
                                n_estimators=n_estimators,
                                min_child_samples=min_child_samples, verbosity=-1,
                                random_state=42)
        
        # Training
        try:
            fitted_model = model.fit(X_train, y_train)
            predictions = fitted_model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, predictions))
        
        except Exception as e:
            logging.info("Echec de l'entrainement du modèle%s", e)
            logging.debug("Détails complets :\n%s", traceback.format_exc())
            raise

        return rmse


    # Recherche des HP et prédictions
    try:
        study = optuna.create_study(direction="minimize")
        study.optimize(objective_lightgbm, n_trials=num_trials, n_jobs=4)
    
    except Exception as e:
        logging.error(
        "Échec de la recherche d’HP. Paramètres : n_trials=%d, n_jobs=%d. Erreur : %s",
        num_trials, 4, str(e),
        exc_info=True)
        raise
    
    # Set optimal score/HP
    dict_best_params = study.best_params
    best_rmse = study.best_value

    return best_rmse, dict_best_params

In [14]:
def inner_cv_lightgbm(outer_id: int, inner_folds: Dict, num_trials: int, inner_n_splits: int) -> Dict:
    
    """Renvoie la liste optimale d'HP en effectuant une cross_validation interne 
    sur le fold (outer_id), avec un nombre d'essais (num_trials)

    Args:
        outer_id (int): ID du fold externe étudié
        inner_folds (Dict): Folds internes
        num_trials (int): Nombre d'essais Optuna
        inner_n_split (int): Nombre de split dans tes outer splits

    Returns:
        fold_best_params (Dict) : Liste optimale d'HP pour le fold outer_id
    """

    #Initialisation
    inner_scores = []
    inner_params = []
    n_folds = inner_n_splits

    # CV interne
    for inner_id in range(n_folds):
        X_train_inner, y_train_inner = inner_folds[f'train_{outer_id}_inner_train_{inner_id}']
        X_val_inner, y_val_inner = inner_folds[f'train_{outer_id}_inner_val_{inner_id}']

        best_rmse, best_params = train_lightgbm(X_train=X_train_inner, 
                                                y_train=y_train_inner, 
                                                X_val=X_val_inner, 
                                                y_val=y_val_inner,
                                                num_trials=num_trials)
        # Scoring sur la CV interne
        inner_scores.append(best_rmse)
        inner_params.append(best_params)

    # HP optimaux pour le outer fold concerné
    best_inner_idx = np.argmin(inner_scores)
    fold_best_params = inner_params[best_inner_idx]

    return fold_best_params

In [15]:
def nested_cv_lightgbm(X : pd.DataFrame, y: pd.Series, num_trials: int, outer_n_splits: int = 5, inner_n_splits: int = 3) -> tuple:
    """Renvoie le RMSE moyen de l'ensemble des folds sur un modèle LightGBM, avec nombre d'essais d'HP.

    Args:
        X (pd.DataFrame): Dataset d'entrainement
        y (pd.DataFrame): Variable cible
        num_trials (int): Nombre d'essais Optuna
        outer_n_splits (int): Nombre de split dans ton dataset X
        inner_n_split (int): Nombre de split dans tes outer splits

    Returns:
        outer_scores, mean_outer_score (tuple) : Liste de l'ensemble des scores 
        des différents folds externe - Moyenne de l'ensemble
    """
    # Initialisation
    outer_folds = outer_time_series_fold(X, y, outer_n_splits=outer_n_splits)
    inner_folds = inner_time_series_fold(outer_folds, inner_n_splits=inner_n_splits)
    outer_scores = []

    for outer_id in range(outer_n_splits):
        
        # Inner CV
        X_train_outer, y_train_outer = outer_folds[f"train_{outer_id}"]
        X_val_outer, y_val_outer = outer_folds[f"val_{outer_id}"]
        fold_best_params = inner_cv_lightgbm(outer_id=outer_id, 
                                             inner_folds=inner_folds,
                                               num_trials=num_trials, 
                                               inner_n_splits=inner_n_splits)

        # Best outer model
        best_model = lgb.LGBMRegressor(**fold_best_params, verbosity=-1, random_state=42)
        fitted_model = best_model.fit(X=X_train_outer, y=y_train_outer)
        
        # Scoring
        y_pred_outer = fitted_model.predict(X_val_outer)
        outer_rmse = np.sqrt(mean_squared_error(y_val_outer, y_pred_outer))
        outer_scores.append(outer_rmse)
    
    # Scoring final
    mean_outer_score = np.mean(outer_scores)
    logging.info(f'RMSE global moyen attendu en production : {mean_outer_score}')
    
    return outer_scores, mean_outer_score

In [ ]:
# Normalization
normalized_solar_ts = df["solar_mw"]/df_power["chronique_capacity"]
normalized_solar_ts = normalized_solar_ts.dropna()
X = df.drop(columns="solar_mw")
y = normalized_solar_ts

# Nested CV
outer_n_splits = 5
inner_n_splits = 2
num_trials = 30
outer_scores, mean_outer_score = nested_cv_lightgbm(X=X, 
                                                    y=y, 
                                                    num_trials=num_trials, 
                                                    outer_n_splits=outer_n_splits, 
                                                    inner_n_splits=inner_n_splits)

[I 2025-08-18 19:54:49,706] A new study created in memory with name: no-name-5b10c5c0-b31f-41a8-82ba-168fbe5b2118
[I 2025-08-18 19:54:55,560] Trial 2 finished with value: 0.07203506342219206 and parameters: {'num_leaves': 45, 'max_depth': 10, 'learning_rate': 0.04196287072294732, 'n_estimators': 193, 'min_child_samples': 27}. Best is trial 2 with value: 0.07203506342219206.
[I 2025-08-18 19:54:55,708] Trial 3 finished with value: 0.0851073013503694 and parameters: {'num_leaves': 49, 'max_depth': 7, 'learning_rate': 0.008127588957913686, 'n_estimators': 252, 'min_child_samples': 31}. Best is trial 2 with value: 0.07203506342219206.
[I 2025-08-18 19:54:56,483] Trial 10 finished with value: 0.07494824325406045 and parameters: {'num_leaves': 25, 'max_depth': 13, 'learning_rate': 0.0767098680559528, 'n_estimators': 272, 'min_child_samples': 46}. Best is trial 2 with value: 0.07203506342219206.
[I 2025-08-18 19:54:57,046] Trial 9 finished with value: 0.07256084348583083 and parameters: {'num

### Modèle LSTM

In [ ]:
# En construction 
class Encoder(nn.Module):
    def __init__(self, input_size: int,
                 hidden_size: int,
                 seq_length: int, 
                 num_layers: int, 
                 dropout: int
                 ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.num_layers = num_layers
        self.dropout = dropout

        self.projection = nn.Linear(input_size, hidden_size)
        self.encoder = nn.LSTM(input_size=self.hidden_size,
                               hidden_size=self.hidden_size,
                               num_layers=self.num_layers,
                               batch_first=True, 
                               dropout=self.dropout)
        
    def forward(self, x, hidden=None):
        """
        x: (batch_size, seq_length, input_size)
        hidden: optional tuple (h0, c0) of shape (num_layers, batch_size, hidden_size)
        """
        batch_size = x.size(0)
        device = x.device

        # Projection
        x = self.projection(x)  # (B, T, hidden_size)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
        
        # Sortie
        out, (hn, cn) = self.encoder(x, (h0, c0))
        
        return out, (hn, cn)
        
#%%
class Decoder(nn.Module):
    def __init__(self, input_size: int,
                hidden_size: int,
                out_seq_length: int, 
                batch_size: int,
                num_layers: int,
                dropout: int,
                output_size: int
                ):
        
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_seq_length = out_seq_length
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.output_size = output_size

        self.lstm = nn.LSTM(input_size=self.input_size,
                            hidden_size=self.hidden_size,
                            num_layers=self.num_layers,
                            batch_first=True,
                            dropout=self.dropout)
        
        self.fc = nn.Linear(in_features=self.hidden_size,
                            out_features=self.output_size)

    def forward(self, x, cn, hn):
        out, (hn, cn) = self.lstm(x, (hn, cn))
        out = self.fc(out)
        
        return out